### 라마3 모델(양자화), 토크나이저 로드

In [15]:
!pip install -q -U bitsandbytes
!pip install -q -U git+https://github.com/huggingface/transformers.git
!pip install -q -U git+https://github.com/huggingface/peft.git
!pip install -q -U git+https://github.com/huggingface/accelerate.git

  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done


In [ ]:
pip install langchain-core langchain-community

In [17]:
import torch
import numpy as np
import pickle
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig, pipeline
from sentence_transformers import SentenceTransformer
from langchain.llms import HuggingFacePipeline
from langchain_core.prompts import PromptTemplate
from langchain.chains import LLMChain
from langchain.vectorstores import FAISS
from langchain.docstore.document import Document

In [ ]:
# 1. LLaMA3 모델 및 토크나이저 로드 (4bit 양자화)
import os

model_name = "meta-llama/Llama-3.1-8B-Instruct"
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_use_double_quant=True,
    bnb_4bit_compute_dtype="bfloat16"
)
tokenizer = AutoTokenizer.from_pretrained(model_name, token = os.getenv("HF_KEY"))
model = AutoModelForCausalLM.from_pretrained(
    model_name, token = os.getenv("HF_KEY"),
    quantization_config=bnb_config,
    device_map="auto",
    torch_dtype="auto"
)
text_generation_pipeline = pipeline(
    model=model,
    tokenizer=tokenizer,
    task="text-generation",
    return_full_text=False,
    max_new_tokens=512
)
llm = HuggingFacePipeline(pipeline=text_generation_pipeline)

tokenizer_config.json:   0%|          | 0.00/55.4k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/9.09M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/296 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/855 [00:00<?, ?B/s]

model.safetensors.index.json:   0%|          | 0.00/23.9k [00:00<?, ?B/s]

...

model-00003-of-00004.safetensors:   0%|          | 0.00/4.92G [00:00<?, ?B/s]

model-00004-of-00004.safetensors:   0%|          | 0.00/1.17G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/184 [00:00<?, ?B/s]

Device set to use cuda:0
<ipython-input-20-8e0c43b1b196>:24: LangChainDeprecationWarning: The class `HuggingFacePipeline` was deprecated in LangChain 0.0.37 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFacePipeline``.
  llm = HuggingFacePipeline(pipeline=text_generation_pipeline)


In [24]:
from langchain.embeddings import HuggingFaceEmbeddings

# 1. 임베딩 모델 래퍼로 감싸기
embedding_model = HuggingFaceEmbeddings(model_name="snunlp/KR-SBERT-V40K-klueNLI-augSTS")

<ipython-input-24-078a869c37f5>:4: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  embedding_model = HuggingFaceEmbeddings(model_name="snunlp/KR-SBERT-V40K-klueNLI-augSTS")


In [30]:
!pip install faiss-cpu

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 31.3/31.3 MB 17.2 MB/s eta 0:00:00


In [26]:
# 3. 뉴스 데이터 준비
news_texts = [
    "2023년 1월 금리 인상으로 인해 주식 시장 변동성 확대됨.",
    "2023년 3월 주요 기업들이 분기 실적 발표 후 주가 상승.",
    "2024년 2월 경제 지표 호조로 투자 심리 개선.",
    "2024년 4월 인플레이션 둔화 기대감으로 시장 안정화.",
    "2024년 5월 글로벌 지정학적 리스크 증가.",
    "2024년 6월 한국은행 기준금리 동결 결정.",
]
documents = [Document(page_content=text) for text in news_texts]
doc_texts = [doc.page_content for doc in documents]

In [31]:
# 4. FAISS 벡터DB 구축
vector_db = FAISS.from_texts(doc_texts, embedding_model)

# 5. 프롬프트 템플릿
prompt_template = """<|begin_of_text|><|start_header_id|>system<|end_header_id|>
당신은 주식 투자자를 위한 뉴스 기반 정보 어시스턴트 챗봇 '뉴스토스'입니다.
당신의 임무는 실시간 뉴스와 과거 유사사건 뉴스 데이터를 바탕으로,
- 사용자의 투자 판단에 도움이 되는 정보를 제공하고,
- 과거 유사사건, 해당 시기의 주가 흐름, 관련 리포트의 핵심 내용을 구체적으로 인용하며,
- 미래 전망 질문에는 과거 사례를 근거로 신중하게 의견을 제시하는 것입니다.

답변 작성 시 반드시 다음을 지켜주세요:
1. 답변 내용 중 포함되는 과거 유사사건의 날짜, 사건명, 당시 주가 흐름(상승/하락/횡보 등), 주요 리포트 내용은 구체적으로 인용하세요.
2. 미래 전망 질문에는 과거 유사사건을 근거로 논리적인 전망을 제시하세요.
3. 답변 마지막에는 '⭐️투자 판단은 본인의 책임입니다.⭐️'라는 안내문을 추가하세요.
4. 답변은 반드시 한글로, 명확하고 간결하게 작성하세요.
5. 제공된 검색 결과(유사도 높은 과거 뉴스, 주가 데이터, 리포트 등)만 근거로 사용하세요. 근거가 없으면 '근거가 없는데 답변해도 될까? 이건 너의 소중한 돈이 걸린 문제야'라고 하세요.
<|eot_id|><|start_header_id|>user<|end_header_id|>
검색 결과: {context}
질문: {question}<|eot_id|><|start_header_id|>assistant<|end_header_id|>"""
prompt = PromptTemplate(input_variables=["context", "question"], template=prompt_template)
llm_chain = LLMChain(llm=llm, prompt=prompt)

<ipython-input-31-573ecdc19479>:22: LangChainDeprecationWarning: The class `LLMChain` was deprecated in LangChain 0.1.17 and will be removed in 1.0. Use :meth:`~RunnableSequence, e.g., `prompt | llm`` instead.
  llm_chain = LLMChain(llm=llm, prompt=prompt)


In [33]:
from transformers import AutoModelForSequenceClassification

# 6. 리트리버 구성
retriever = vector_db.as_retriever(search_type="similarity", search_kwargs={"k": 3})

# 7. 리랭커(Cross-Encoder) 모델 : 검색 결과 재정렬용 모델
reranker_tokenizer = AutoTokenizer.from_pretrained('BAAI/bge-reranker-v2-m3')
reranker_model = AutoModelForSequenceClassification.from_pretrained('BAAI/bge-reranker-v2-m3').to("cuda" if torch.cuda.is_available() else "cpu")
reranker_model.eval()

config.json:   0%|          | 0.00/795 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/2.27G [00:00<?, ?B/s]

XLMRobertaForSequenceClassification(
  (roberta): XLMRobertaModel(
    (embeddings): XLMRobertaEmbeddings(
      (word_embeddings): Embedding(250002, 1024, padding_idx=1)
      (position_embeddings): Embedding(8194, 1024, padding_idx=1)
      (token_type_embeddings): Embedding(1, 1024)
      (LayerNorm): LayerNorm((1024,), eps=1e-05, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): XLMRobertaEncoder(
      (layer): ModuleList(
        (0-23): 24 x XLMRobertaLayer(
          (attention): XLMRobertaAttention(
            (self): XLMRobertaSdpaSelfAttention(
              (query): Linear(in_features=1024, out_features=1024, bias=True)
              (key): Linear(in_features=1024, out_features=1024, bias=True)
              (value): Linear(in_features=1024, out_features=1024, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): XLMRobertaSelfOutput(
              (dense): Linear(in_features=1024, o

In [37]:
# 검색된 문서 후보군(top-k)을 딥러닝 기반 리랭커(Cross-Encoder)로 재정렬
def rerank_docs(docs, question): # 벡터 DB에서 1차로 추출된 문서 리스트와 질문이 들어가
    pairs = [(question, doc.page_content) for doc in docs] # 각 문서랑 질문을 한 쌍으로 묶어
    # Cross-Encoder 입력 포맷: [query] [SEP] [doc]
    inputs = reranker_tokenizer([f"{q} [SEP] {d}" for q, d in pairs], return_tensors="pt", padding=True, truncation=True).to(reranker_model.device) # cross-encoder 모델 입력 포맷, 텍스트를 토큰으로 변환해 Pytorch 텐서로 만들어
    with torch.no_grad():
        scores = reranker_model(**inputs).logits.squeeze().cpu().numpy() # 각 질문-문서 쌍의 관련성 점수를 계산해
    reranked = sorted(zip(docs, scores), key=lambda x: x[1], reverse=True) # 점수 높은 문서가 앞에 오도록 정렬해
        # 점수와 문서 함께 출력
    print("\n=== 리랭킹 모델 점수 ===")
    for doc, score in zip(docs, scores):
      print(f"Score: {score:.4f} | {doc.page_content}")
    return [doc for doc, _ in reranked]

# rag_pipeline 함수
def rag_pipeline(question):
    print(f"\n=== 사용자 질문 ===\n{question}")

    # 1. Retrieve
    docs = retriever.get_relevant_documents(question)
    print("\n=== 1차 검색 결과 ===")
    for idx, doc in enumerate(docs, 1):
        print(f"[Doc {idx}] {doc.page_content}")

    # 2. Re-rank
    reranked_docs = rerank_docs(docs, question)
    print("\n=== 재정렬 후 결과 ===")
    for idx, doc in enumerate(reranked_docs, 1):
        print(f"[Rank {idx}] {doc.page_content}")

    # 3. Context 생성
    context = "\n\n".join([doc.page_content for doc in reranked_docs])
    print("\n=== LLM 입력 컨텍스트 ===")
    print(context)

    # 4. LLM 답변 생성
    answer = llm_chain.run({"context": context, "question": question})
    return {
        "answer": answer,
        "reranked_docs": reranked_docs
    }

In [40]:
# 실행 예시
if __name__ == "__main__":
    query = "금리 인하 예정"
    result = rag_pipeline(query)

    print("\n=== 최종 참고 문서 ===")
    for idx, doc in enumerate(result["reranked_docs"], 1):
        print(f"{idx}. {doc.page_content}")

    print("\n=== LLM 최종 답변 ===")
    print(result["answer"])

Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.



=== 사용자 질문 ===
금리 인하 예정

=== 1차 검색 결과 ===
[Doc 1] 2024년 6월 한국은행 기준금리 동결 결정.
[Doc 2] 2023년 1월 금리 인상으로 인해 주식 시장 변동성 확대됨.
[Doc 3] 2024년 4월 인플레이션 둔화 기대감으로 시장 안정화.

=== 리랭킹 모델 점수 ===
Score: -1.3845 | 2024년 6월 한국은행 기준금리 동결 결정.
Score: -3.9790 | 2023년 1월 금리 인상으로 인해 주식 시장 변동성 확대됨.
Score: -3.3009 | 2024년 4월 인플레이션 둔화 기대감으로 시장 안정화.

=== 재정렬 후 결과 ===
[Rank 1] 2024년 6월 한국은행 기준금리 동결 결정.
[Rank 2] 2024년 4월 인플레이션 둔화 기대감으로 시장 안정화.
[Rank 3] 2023년 1월 금리 인상으로 인해 주식 시장 변동성 확대됨.

=== LLM 입력 컨텍스트 ===
2024년 6월 한국은행 기준금리 동결 결정.

2024년 4월 인플레이션 둔화 기대감으로 시장 안정화.

2023년 1월 금리 인상으로 인해 주식 시장 변동성 확대됨.

=== 최종 참고 문서 ===
1. 2024년 6월 한국은행 기준금리 동결 결정.
2. 2024년 4월 인플레이션 둔화 기대감으로 시장 안정화.
3. 2023년 1월 금리 인상으로 인해 주식 시장 변동성 확대됨.

=== LLM 최종 답변 ===


안녕하세요! 뉴스토스입니다.

2024년 6월 한국은행이 기준금리 동결을 결정했습니다. 이 결정은 2024년 4월에 인플레이션 둔화 기대감으로 시장 안정화에 기여했습니다. 이전에, 2023년 1월 금리 인상으로 인해 주식 시장 변동성이 확대되었습니다.

금리 인하 예정에 대한 질문에 대한 답변으로, 금리 인하로 인해 주식 시장은 일반적으로 상승할 가능성이 있지만, 과거 사례를 보면 다음과 같이 예상할 수 있습니다.

과거 유사사건으로는 2018년 7월 한국은행이 기준금리를 인하한 경우가 있습니다. 당

# llama3.1 모델 경량화 버전 다운로드

In [5]:
from huggingface_hub import hf_hub_download
import os

In [6]:
# --- 설정 부분: bartowski 모델 정보로 변경 ---
repo_id = "bartowski/Meta-Llama-3.1-8B-Instruct-GGUF"
filename = "Meta-Llama-3.1-8B-Instruct-Q4_K_M.gguf" # <-- 선택한 파일명으로 변경 (예: Q4_K_M)

# 모델을 저장할 로컬 경로 (프로젝트 루트 기준으로 작성)
local_dir = "/Users/JooAnLee/final_project/models/quantized_llama3.1"

os.makedirs(local_dir, exist_ok=True)  # 디렉토리 있으면 넘어가고 없으면 만들어

print(f"Downloading '{filename}' from '{repo_id}' to '{local_dir}'...")
try:
    hf_hub_download(
        repo_id=repo_id,
        filename=filename,
        local_dir=local_dir,
        local_dir_use_symlinks=False
    )
    print("Download complete!")
    print(f"Model saved at: {os.path.join(local_dir, filename)}")
except Exception as e:
    print(f"Error during download: {e}")
    print("Please check repo_id, filename, and your internet connection.")
    print("Also ensure you have 'huggingface_hub' installed: pip install huggingface_hub")

Download complete!
Model saved at: /Users/JooAnLee/final_project/models/quantized_llama3.1/Meta-Llama-3.1-8B-Instruct-Q4_K_M.gguf


In [4]:
import llama_cpp
print(llama_cpp.__version__)  # 0.2.72 이상 권장

ModuleNotFoundError: No module named 'llama_cpp'

## models/rag_pipeline.py 코드 test (1차)

In [ ]:
import torch
import numpy as np
import pickle
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig, pipeline
from langchain.llms import HuggingFacePipeline
from langchain_core.prompts import PromptTemplate
from langchain.chains import LLMChain
from langchain.vectorstores import FAISS
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.docstore.document import Document
from transformers import AutoModelForSequenceClassification


class NewsTossChatbot:
    def __init__(
        self,
        llama_model_name: "meta-llama/Llama-3.1-8B-Instruct",
        llama_token: HF_KEY,  # .env로 관리하거나... 추후 없애야함
        embedding_model_name: "snunlp/KR-SBERT-V40K-klueNLI-augSTS",  # 임베딩모델 확인 필요
        reranker_model_name: "'BAAI/bge-reranker-v2-m3'",  # 딥러닝 회귀 모델_path 로 수정 필요
        news_texts: ["2023년 1월 금리 인상으로 인해 주식 시장 변동성 확대됨.", "2023년 3월 주요 기업들이 분기 실적 발표 후 주가 상승.",
                    "2024년 2월 경제 지표 호조로 투자 심리 개선.", "2024년 4월 인플레이션 둔화 기대감으로 시장 안정화.",
                    "2024년 5월 글로벌 지정학적 리스크 증가.", "2024년 6월 한국은행 기준금리 동결 결정."],
        k: int = 5
    ):
        
        # 1. LLaMA3 LLM 로드
        bnb_config = BitsAndBytesConfig(
            load_in_4bit=True,
            bnb_4bit_quant_type="nf4",
            bnb_4bit_use_double_quant=True,
            bnb_4bit_compute_dtype="bfloat16"
        )
        tokenizer = AutoTokenizer.from_pretrained(llama_model_name, token=llama_token)
        model = AutoModelForCausalLM.from_pretrained(
            llama_model_name,
            token=llama_token,
            quantization_config=bnb_config,
            device_map="auto",
            torch_dtype="auto"
        )
        text_generation_pipeline = pipeline(
            model=model,
            tokenizer=tokenizer,
            task="text-generation",
            return_full_text=False,
            max_new_tokens=512
        )
        self.llm = HuggingFacePipeline(pipeline=text_generation_pipeline)

        # 2. 임베딩 모델 및 벡터DB
        self.embedding_model = HuggingFaceEmbeddings(model_name=embedding_model_name)
        self.documents = [Document(page_content=text) for text in news_texts]
        doc_texts = [doc.page_content for doc in self.documents]
        self.vector_db = FAISS.from_texts(doc_texts, self.embedding_model)
        self.k = k

        # 3. Cross-Encoder 리랭커 모델
        self.reranker_tokenizer = AutoTokenizer.from_pretrained(reranker_model_name)
        self.reranker_model = AutoModelForSequenceClassification.from_pretrained(reranker_model_name).to(
            "cuda" if torch.cuda.is_available() else "cpu"
        )
        self.reranker_model.eval()

        # 4. LLMChain & 프롬프트
        prompt_template = """<|begin_of_text|><|start_header_id|>system<|end_header_id|>
        당신은 주식 투자자를 위한 뉴스 기반 정보 어시스턴트 챗봇 '뉴스토스'입니다.
        당신의 임무는 실시간 뉴스와 과거 유사사건 뉴스 데이터를 바탕으로,
        - 사용자의 투자 판단에 도움이 되는 정보를 제공하고,
        - 과거 유사사건, 해당 시기의 주가 흐름, 관련 리포트의 핵심 내용을 구체적으로 인용하며,
        - 미래 전망 질문에는 과거 사례를 근거로 신중하게 의견을 제시하는 것입니다.

        답변 작성 시 반드시 다음을 지켜주세요:
        1. 답변 내용 중 포함되는 과거 유사사건의 날짜, 사건명, 당시 주가 흐름(상승/하락/횡보 등), 주요 리포트 내용은 구체적으로 인용하세요.
        2. 미래 전망 질문에는 과거 유사사건을 근거로 논리적인 전망을 제시하세요.
        3. 답변 마지막에는 '⭐️투자 판단은 본인의 책임입니다.⭐️'라는 안내문을 추가하세요.
        4. 답변은 반드시 한글로, 명확하고 간결하게 작성하세요.
        5. 제공된 검색 결과(유사도 높은 과거 뉴스, 주가 데이터, 리포트 등)만 근거로 사용하세요. 근거가 없으면 '근거가 없는데 답변해도 될까? 이건 너의 소중한 돈이 걸린 문제야'라고 하세요.
        <|eot_id|><|start_header_id|>user<|end_header_id|>
        검색 결과: {context}
        질문: {question}<|eot_id|><|start_header_id|>assistant<|end_header_id|>"""
        prompt = PromptTemplate(input_variables=["context", "question"], template=prompt_template)
        self.llm_chain = LLMChain(llm=self.llm, prompt=prompt)

    def retrieve_docs(self, query):
        retriever = self.vector_db.as_retriever(search_type="similarity", search_kwargs={"k": self.k})
        return retriever.get_relevant_documents(query)

    def rerank_docs(self, query, docs):
        pairs = [(query, doc.page_content) for doc in docs]
        inputs = self.reranker_tokenizer(
            [f"{q} [SEP] {d}" for q, d in pairs],
            return_tensors="pt",
            padding=True,
            truncation=True
        ).to(self.reranker_model.device)
        with torch.no_grad():
            scores = self.reranker_model(**inputs).logits.squeeze().cpu().numpy()
        reranked = sorted(zip(docs, scores), key=lambda x: x[1], reverse=True)
        return [doc for doc, _ in reranked]

    def answer(self, query):
        docs = self.retrieve_docs(query)
        reranked_docs = self.rerank_docs(query, docs)
        context = "\n\n".join([doc.page_content for doc in reranked_docs])
        answer = self.llm_chain.run({"context": context, "question": query})
        return {
            "answer": answer,
            "reranked_docs": reranked_docs
        }
